**Задание Pro**

Проведите ряд экспериментов с архитектурой нейронной сети:

1. Исследуйте влияние количества слоев conv2_d.
2. Исследуйте влияние количества нейронов.
3. Исследуйте влияние MaxPooling2D в сети.
4. Исследуйте влияние Dropout.
5. Напишите выводы по результатам проведенных тестов.

Задание получилось очень длинным.

Заранее у проверяющего преподавателя извиняюсь. 

Очень много гиперпараметров для перебора. В конце имеется общая таблица по всем вариантам сетей. 

Так же после общей таблицы идет топ-10 лучших и худших результата. Чтобы не искать их в большой таблице.

Каждое задание можно скрыть.

Для того что бы код не был еще длинее, параметр verbose установлен в 0 (без просмотра обучения), так как количество эпох 30.

In [ ]:
# Для начала импортируем все необходимые для создания нейронной сети библиотеки.

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
import random
import math
import os
from google.colab import drive
from sklearn.model_selection import train_test_split

%matplotlib inline

# Домашние задание будет выполняться на примерах работы с Cifar100

# Поготовка данных 

In [ ]:
# Для начала загрузим базу cifar100
(x_train, y_train),(x_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 4s 0us/step


In [ ]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)


(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [ ]:
# Разделим обучающую выборку на (обучающую и проверочную). Чтобы не задействовать тестовую во время обучения
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)
# Посмотрим на размр выборок
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
# Преобразуем ответы y_train, y_val и y_test в ohe
y_train = utils.to_categorical (y_train,100)
y_val = utils.to_categorical (y_val,100)
y_test = utils.to_categorical (y_test,100)

# Посмотрим на размерность получившихся массивов 
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)


(40000, 100)
(10000, 100)
(10000, 100)


In [ ]:
# Создадим таблицу. Столбцы будут иметь название: Количество нейронов в слоях Conv2D, ошибки и точности на всех выборках:
# learn - обучающая, val - проверочная, test - тестовая. 
# В качестве названия строк парметры которые были изменены. 

df = pd.DataFrame(columns = ['neurons in Conv2D', 'learn_loss', 'learn_accuracy', 'val_loss', 'val_accuracy', 'test_loss', 'test_accuracy'])
df

,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy


# Задание 1. Исследуйте влияние количества слоев conv2_d

В данном задании будут варианты:

 - Модель из лекции с 6 слоями Сonv2d (на 32, 64 и 256 фильтров)
 - А так же будет сеть с 1, 2, 3, 6, 9 слоями Сonv2d и без них. 

Для начала создадим функции:
 - Моделей нейронных сетей для данного задания (**model_1(), model_2(), model_3()**)
 - Функцию обучения **train_network()**

## Функция обучения 

In [ ]:
batch_size = 150

# Создадим функцию обучения сети
def train_network(model):
  #Компилируем сеть
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  #Обучаем сеть на данных cifar100 и сохраняем результат в переменной history
  history = model.fit(x_train, y_train,  batch_size= batch_size, epochs=30, validation_data=(x_val, y_val), verbose=0)

  # Делаем проверку на тестовой выборке
  scores = model.evaluate(x_test, y_test, verbose=0)

  # Создаем пустой словарь. 
  dct = []  
  # В цикле проходимся по всем значениям из history                                                  
  for v in history.history.values():                                        
    dct.append (v)                                            # Результат выгружаются в словарь.                                                                
  result = np.array(dct)                                      # Переводим в numpy массив.
  return result, scores                                       # Функция возвращает результат обучения

## Модели нейронных сетей (6 функций)

In [ ]:
# Модель с 3 слоями Conv2D
def model_1():
  model_1 = Sequential()
  model_1.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_1.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_1.add(MaxPooling2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.25))

  model_1.add(BatchNormalization())
  model_1.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_1.add(MaxPooling2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.25))

  model_1.add(BatchNormalization())
  model_1.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_1.add(MaxPooling2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.25))

  model_1.add(Flatten())
  model_1.add(Dense(512, activation='relu'))
  model_1.add(Dropout(0.25))
  model_1.add(Dense(100, activation='softmax'))

  return model_1

# Модель с 6 слоями Conv2D
def model_2():
  model_2 = Sequential()
  model_2.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_2.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_2.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_2.add(MaxPooling2D(pool_size=(2, 2)))
  model_2.add(Dropout(0.25))

  model_2.add(BatchNormalization())
  model_2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_2.add(MaxPooling2D(pool_size=(2, 2)))
  model_2.add(Dropout(0.25))

  model_2.add(BatchNormalization())
  model_2.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_2.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_2.add(MaxPooling2D(pool_size=(2, 2)))
  model_2.add(Dropout(0.25))

  model_2.add(Flatten())
  model_2.add(Dense(512, activation='relu'))
  model_2.add(Dropout(0.25))
  model_2.add(Dense(100, activation='softmax'))

  return model_2


# Модель с 9 слоями Conv2D
def model_3():
  model_3 = Sequential()
  model_3.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_3.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_3.add(MaxPooling2D(pool_size=(2, 2)))
  model_3.add(Dropout(0.25))

  model_3.add(BatchNormalization())
  model_3.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_3.add(MaxPooling2D(pool_size=(2, 2)))
  model_3.add(Dropout(0.25))

  model_3.add(BatchNormalization())
  model_3.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_3.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model_3.add(MaxPooling2D(pool_size=(2, 2)))
  model_3.add(Dropout(0.25))

  model_3.add(Flatten())
  model_3.add(Dense(512, activation='relu'))
  model_3.add(Dropout(0.25))
  model_3.add(Dense(100, activation='softmax'))

  return model_3

# Оставим только 2 слоя Conv2D
def model_4():
  model_4 = Sequential()
  model_4.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_4.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_4.add(MaxPooling2D(pool_size=(2, 2)))
  model_4.add(Dropout(0.25))

  model_4.add(BatchNormalization())
  model_4.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model_4.add(MaxPooling2D(pool_size=(2, 2)))
  model_4.add(Dropout(0.25))

  model_4.add(BatchNormalization())
  model_4.add(MaxPooling2D(pool_size=(2, 2)))
  model_4.add(Dropout(0.25))

  model_4.add(Flatten())
  model_4.add(Dense(512, activation='relu'))
  model_4.add(Dropout(0.25))
  model_4.add(Dense(100, activation='softmax'))

  return model_4


# Оставим 1 слой Conv2D
def model_5():
  model_5 = Sequential()
  model_5.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_5.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model_5.add(MaxPooling2D(pool_size=(2, 2)))
  model_5.add(Dropout(0.25))

  model_5.add(BatchNormalization())
  model_5.add(MaxPooling2D(pool_size=(2, 2)))
  model_5.add(Dropout(0.25))

  model_5.add(BatchNormalization())
  model_5.add(MaxPooling2D(pool_size=(2, 2)))
  model_5.add(Dropout(0.25))

  model_5.add(Flatten())
  model_5.add(Dense(512, activation='relu'))
  model_5.add(Dropout(0.25))
  model_5.add(Dense(100, activation='softmax'))

  return model_5

# Без слоев Conv2D
def model_6():
  model_6 = Sequential()
  model_6.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_6.add(MaxPooling2D(pool_size=(2, 2)))
  model_6.add(Dropout(0.25))

  model_6.add(BatchNormalization())
  model_6.add(MaxPooling2D(pool_size=(2, 2)))
  model_6.add(Dropout(0.25))

  model_6.add(BatchNormalization())
  model_6.add(MaxPooling2D(pool_size=(2, 2)))
  model_6.add(Dropout(0.25))

  model_6.add(Flatten())
  model_6.add(Dense(512, activation='relu'))
  model_6.add(Dropout(0.25))
  model_6.add(Dense(100, activation='softmax'))

  return model_6

## Обучение разных моделей с занесением данных в таблицу

In [ ]:
# Обучение 1 модели
model_1 = model_1()
result, scores = train_network(model_1)
df.loc['3 layers conv2D'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 2 модели
model_2 = model_2()
result, scores = train_network(model_2)
df.loc['6 layers conv2D'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 3 модели
model_3 = model_3()
result, scores = train_network(model_3)
df.loc['9 layers conv2D'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 4 модели (2 слоя conv2D)
model_4 = model_4()
result, scores = train_network(model_4)
df.loc['2 layers conv2D'] = ['32-64', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 5 модели (1 слой conv2D)
model_5 = model_5()
result, scores = train_network(model_5)
df.loc['1 layers conv2D'] = ['32', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 6 модели (без слоев conv2D)
model_6 = model_6()
result, scores = train_network(model_6)
df.loc['0 layers conv2D'] = ['0', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Задача 2. Исследуйте влияние количества нейронов

**Изменять будем кол-во фильтров(нейронов) в слоях Conv2D**

Изменения нейронов в Dense слое было в прошлом задании Lite. Здесь уже менять этот параметр не будем. 

В этом задании будет очень много вариантов, поэтому будем использовать цикл.
- 3, 6, 9 слоев Conv2D
- и каждом по 27 вариантов

In [ ]:
# Для начала создадим списки, в которых будут перебираться варианты количества нейронов
neurons_1 = [4, 8, 32]
neurons_2 = [8, 16, 64]
neurons_3 = [16, 32, 128]

# И счетчик для таблицы, чтобы были новые строки 
count = 1

## Вариант с 3 слоями Conv2D

In [ ]:
# Вариант с 3 слоями Conv2D
for i in neurons_1:
  for j in neurons_2:
    for n in neurons_3:
        model = Sequential()
        model.add(BatchNormalization(input_shape=(32, 32, 3)))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(j, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(n, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(100, activation='softmax'))

        result, scores = train_network(model)
        df.loc['v{} - 3 layers conv2D'.format(count)] = ['{}-{}-{}'.format(i, j, n), result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]
        count += 1

## Вариант с 6 слоями Conv2D

In [ ]:
# Вариант с 6 слоями Conv2D
for i in neurons_1:
  for j in neurons_2:
    for n in neurons_3:
        model = Sequential()
        model.add(BatchNormalization(input_shape=(32, 32, 3)))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(j, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(j, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(n, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(n, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(100, activation='softmax'))

        result, scores = train_network(model)
        df.loc['v{} - 6 layers conv2D'.format(count)] = ['{}-{}-{}'.format(i, j, n), result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]
        count += 1

## Вариант с 9 слоями Conv2D

In [ ]:
# Вариант с 9 слоями Conv2D
for i in neurons_1:
  for j in neurons_2:
    for n in neurons_3:
        model = Sequential()
        model.add(BatchNormalization(input_shape=(32, 32, 3)))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(j, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(j, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(BatchNormalization())
        model.add(Conv2D(n, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(i, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(n, (3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(100, activation='softmax'))

        result, scores = train_network(model)
        df.loc['v{} - 9 layers conv2D'.format(count)] = ['{}-{}-{}'.format(i, j, n), result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]
        count += 1

# Задача 3.  Исследуйте влияние MaxPooling2D в сети.

**Изначально было 3 слоя MaxPooling2D с параметром pool_size=(2, 2)**

- В первом варианте будем менять значение pool_size
- В других вариантов будем изменять количество слоев MaxPooling2D (3,2,1,0)

## Изменение значений pool_size

In [ ]:
pl = [2,3]    # Чтобы перебирать ниже в цикле значения pool_size

## Обучение моделей 

In [ ]:
# Модель с 3 слоями MaxPooling2D
for i in pl: 
  model = Sequential()
  model.add(BatchNormalization(input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(100, activation='softmax'))

  result, scores = train_network(model)
  df.loc['3 layers MaxPooling2D, pool_size=({}, {})'.format(i,i)] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]


# Модель с 2 слоями MaxPooling2D
for i in pl: 
  model = Sequential()
  model.add(BatchNormalization(input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(100, activation='softmax'))

  result, scores = train_network(model)
  df.loc['2 layers MaxPooling2D, pool_size=({}, {})'.format(i,i)] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]


# Модель с 1 слоем MaxPooling2D
for i in pl: 
  model = Sequential()
  model.add(BatchNormalization(input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(i, i)))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Dropout(0.25))

  model.add(BatchNormalization())
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(100, activation='softmax'))

  result, scores = train_network(model)
  df.loc['1 layers MaxPooling2D, pool_size=({}, {})'.format(i,i)] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]


# Модель с без слоев MaxPooling2D

model = Sequential()
model.add(BatchNormalization(input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(100, activation='softmax'))

result, scores = train_network(model)
df.loc['0 layers MaxPooling2D, pool_size=({}, {})'.format(i,i)] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Задача 4. Исследуйте влияние Dropout

В данной задаче будут изменяться:
 - Значения слоя Dropout
 - А так же количество слоев Dropout

## Изменения значений Dropout

In [ ]:
dp = [0.2,0.3,0.4,0.5]

In [ ]:
# Перебор значений параметра Dropout
for i in dp: 
  model = Sequential()
  model.add(BatchNormalization(input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(i))

  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(i))

  model.add(BatchNormalization())
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(i))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(i))
  model.add(Dense(100, activation='softmax'))

  result, scores = train_network(model)
  df.loc['Dropout({})'.format(i)] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

## Изменение количества слоев Dropout

In [ ]:
# C 2 слоями Dropout
def model_7():
  model_7 = Sequential()
  model_7.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_7.add(MaxPooling2D(pool_size=(2, 2)))
  model_7.add(Dropout(0.25))

  model_7.add(BatchNormalization())
  model_7.add(MaxPooling2D(pool_size=(2, 2)))
  model_7.add(Dropout(0.25))

  model_7.add(BatchNormalization())
  model_7.add(MaxPooling2D(pool_size=(2, 2)))
  model_7.add(Dropout(0.25))

  model_7.add(Flatten())
  model_7.add(Dense(512, activation='relu'))
  model_7.add(Dense(100, activation='softmax'))

  return model_7

# C 1 слоем Dropout
def model_8():
  model_8 = Sequential()
  model_8.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_8.add(MaxPooling2D(pool_size=(2, 2)))
  model_8.add(Dropout(0.25))

  model_8.add(BatchNormalization())
  model_8.add(MaxPooling2D(pool_size=(2, 2)))

  model_8.add(BatchNormalization())
  model_8.add(MaxPooling2D(pool_size=(2, 2)))

  model_8.add(Flatten())
  model_8.add(Dense(512, activation='relu'))
  model_8.add(Dense(100, activation='softmax'))

  return model_8

# Без слоев Dropout
def model_9():
  model_9 = Sequential()
  model_9.add(BatchNormalization(input_shape=(32, 32, 3)))
  model_9.add(MaxPooling2D(pool_size=(2, 2)))

  model_9.add(BatchNormalization())
  model_9.add(MaxPooling2D(pool_size=(2, 2)))

  model_9.add(BatchNormalization())
  model_9.add(MaxPooling2D(pool_size=(2, 2)))

  model_9.add(Flatten())
  model_9.add(Dense(512, activation='relu'))
  model_9.add(Dense(100, activation='softmax'))

  return model_9

**Обучение сети**

In [ ]:
# Обучение 7 модели
model_7 = model_7()
result, scores = train_network(model_7)
df.loc['2 layers Dropout'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 8 модели
model_8 = model_8()
result, scores = train_network(model_8)
df.loc['1 layer Dropout'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Обучение 9 модели
model_9 = model_9()
result, scores = train_network(model_9)
df.loc['0 layers Dropout'] = ['32-64-256', result[0][-1], result[1][-1]*100, result[2][-1], result[3][-1]*100, scores[0], scores[1]*100]

# Общая таблица значений всех вариантов сетей (102 Варианта)

In [ ]:
df

,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
3 layers conv2D,32-64-256,1.437297,58.887500,2.126698,46.619999,2.039202,48.870000
6 layers conv2D,32-64-256,0.748036,77.265000,2.186973,49.739999,2.180316,49.930000
9 layers conv2D,32-64-256,0.595209,81.732500,2.990374,42.019999,2.977854,42.010000
2 layers conv2D,32-64,1.423554,58.950001,3.849976,37.540001,3.677330,38.769999
1 layers conv2D,32,1.962690,46.987501,2.937804,32.789999,2.884467,33.759999
...,...,...,...,...,...,...,...
Dropout(0.4),32-64-256,1.488037,57.650000,1.902068,50.019997,1.852370,51.450002
Dropout(0.5),32-64-256,1.906626,47.900000,1.918747,47.839999,1.891764,49.280000
2 layers Dropout,32-64-256,3.669994,14.247499,4.568819,4.500000,4.550655,4.900000
1 layer Dropout,32-64-256,3.049628,25.475001,3.952462,11.590000,3.941720,11.930000


**Можно составить топ-3 результатов (лучших и худших). Чтобы было проще ориентироваться.**

In [ ]:
print('10 лучших результата по минимальной ошибке тестовой выборки')
df.sort_values('test_loss').head(10)

10 лучших результата по минимальной ошибке тестовой выборки


,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
Dropout(0.4),32-64-256,1.488037,57.650000,1.902068,50.019997,1.852370,51.450002
Dropout(0.5),32-64-256,1.906626,47.900000,1.918747,47.839999,1.891764,49.280000
"3 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,1.475172,57.854998,1.965703,47.920001,1.934500,49.010000
Dropout(0.3),32-64-256,1.088341,67.962497,1.999400,50.889999,1.968466,51.789999
v52 - 6 layers conv2D,32-64-16,1.860657,47.940001,2.062803,45.460001,2.026954,46.610001
v53 - 6 layers conv2D,32-64-32,1.481047,57.130003,2.065264,46.090001,2.027793,47.560000
3 layers conv2D,32-64-256,1.437297,58.887500,2.126698,46.619999,2.039202,48.870000
v26 - 3 layers conv2D,32-64-32,1.859832,48.867500,2.113978,45.140001,2.064412,45.820001
v27 - 3 layers conv2D,32-64-128,1.600563,55.142498,2.124712,46.079999,2.072597,46.810001
"2 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,1.155089,65.665001,2.097550,48.330000,2.077357,49.349999


In [ ]:
print('10 лучших результата по максимальной точности тестовой выборки')
df.sort_values('test_accuracy', ascending=False).head(10)

10 лучших результата по максимальной точности тестовой выборки


,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
Dropout(0.3),32-64-256,1.088341,67.962497,1.999400,50.889999,1.968466,51.789999
Dropout(0.4),32-64-256,1.488037,57.650000,1.902068,50.019997,1.852370,51.450002
"3 layers MaxPooling2D, pool_size=(2, 2)",32-64-256,0.828951,74.632502,2.158352,50.569999,2.098121,51.289999
6 layers conv2D,32-64-256,0.748036,77.265000,2.186973,49.739999,2.180316,49.930000
Dropout(0.2),32-64-256,0.577412,81.997502,2.627393,48.069999,2.565757,49.439999
"2 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,1.155089,65.665001,2.097550,48.330000,2.077357,49.349999
Dropout(0.5),32-64-256,1.906626,47.900000,1.918747,47.839999,1.891764,49.280000
"3 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,1.475172,57.854998,1.965703,47.920001,1.934500,49.010000
3 layers conv2D,32-64-256,1.437297,58.887500,2.126698,46.619999,2.039202,48.870000
v54 - 6 layers conv2D,32-64-128,0.834345,74.409997,2.283110,47.690001,2.241220,48.300001


In [ ]:
print('10 худших! результата по минимальной ошибке тестовой выборки')
df.sort_values('test_loss', ascending=False).head(10)

10 худших! результата по минимальной ошибке тестовой выборки


,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
"0 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,4.605286,1.070000,4.607408,0.720000,4.605425,1.000000
"0 layers MaxPooling2D, pool_size=(2, 2)",32-64-256,4.605312,1.070000,4.607369,0.720000,4.605417,1.000000
"1 layers MaxPooling2D, pool_size=(2, 2)",32-64-256,4.605293,1.070000,4.607322,0.720000,4.605409,1.000000
2 layers Dropout,32-64-256,3.669994,14.247499,4.568819,4.500000,4.550655,4.900000
0 layers conv2D,0,3.760004,12.695000,4.360786,5.320000,4.348200,5.300000
1 layer Dropout,32-64-256,3.049628,25.475001,3.952462,11.590000,3.941720,11.930000
0 layers Dropout,32-64-256,2.941326,28.022501,3.730120,16.970000,3.722272,17.399999
2 layers conv2D,32-64,1.423554,58.950001,3.849976,37.540001,3.677330,38.769999
v56 - 9 layers conv2D,4-8-32,2.713228,30.739999,3.441798,19.630000,3.411054,20.469999
v63 - 9 layers conv2D,4-64-128,1.910319,47.674999,3.426141,24.400000,3.390957,24.890000


In [ ]:
print('10 худших! результата по минимальной точности тестовой выборки')
df.sort_values('test_accuracy').head(10)

10 худших! результата по минимальной точности тестовой выборки


,neurons in Conv2D,learn_loss,learn_accuracy,val_loss,val_accuracy,test_loss,test_accuracy
"0 layers MaxPooling2D, pool_size=(3, 3)",32-64-256,4.605286,1.070000,4.607408,0.72,4.605425,1.000000
"1 layers MaxPooling2D, pool_size=(2, 2)",32-64-256,4.605293,1.070000,4.607322,0.72,4.605409,1.000000
"0 layers MaxPooling2D, pool_size=(2, 2)",32-64-256,4.605312,1.070000,4.607369,0.72,4.605417,1.000000
2 layers Dropout,32-64-256,3.669994,14.247499,4.568819,4.50,4.550655,4.900000
0 layers conv2D,0,3.760004,12.695000,4.360786,5.32,4.348200,5.300000
1 layer Dropout,32-64-256,3.049628,25.475001,3.952462,11.59,3.941720,11.930000
0 layers Dropout,32-64-256,2.941326,28.022501,3.730120,16.97,3.722272,17.399999
v56 - 9 layers conv2D,4-8-32,2.713228,30.739999,3.441798,19.63,3.411054,20.469999
v57 - 9 layers conv2D,4-8-128,2.468252,35.627499,3.300087,21.85,3.278692,22.560000
v55 - 9 layers conv2D,4-8-16,2.956135,26.019999,3.207178,23.35,3.199759,24.200000


# Вывод:

Чтобы корректно сравнивать разные архитектуры сетей, определенные параметры были без без изменений, а именно: 
  - Одни и теже выборки для всех моделей (обучающая, проверочная и тестовая)
  - loss="categorical_crossentropy"
  - optimizer="adam"
  - batch_size = 150
  - epochs=30

Менялись значения:
 - Kоличество слоев conv2_d.
 - Kоличество нейронов в слоях conv2_d.
 - Kоличество слоев MaxPooling2D
 - Значение pool_size в слоях MaxPooling2D
 - Kоличество слоев Dropout
 - Значение отключаемых нейронов в слоях Dropout


 По результатам проведенной работы монжо сказать что:
  - Лучше всех оказалась сеть с тремя Conv2d(32-64-256), MaxPooling2D и Dropout(0.3)
 - Без использования таких слоев как MaxPooling2D и Dropout нейронная сеть показывает результат значительно хуже.
 - Одного слоя Conv2d и МaxPooling2D не достаточно. С тремя слоями результат значительно лучше
 - Значение слоя Dropout тоже влияет на качество обучения. Слишком большое значение плохо. При слишком малом или его отсутствие может быть переобучение сети. 
 - Изменение параметра pool_size с (2,2) на (3,3) дало результат лучше.  
 - Незначительное изменение количества фильтров у слоя Conv2D не дает заметных результатов (+-2%). Но все же в реальных проектах это показатель за который стоит побороться.
 - Но значительное уменьшение данных фильтров приводит к большему проценту ошибок.  
 - Нахождение оптимальных параметров дело не легкое и требует достаточного количества опыта, навыков, знания языка и имение работать с нужными библиотеками.